In [20]:
from pathlib import Path
from scanpy import read_h5ad
import pandas as pd
import anndata

h5ad_tracker = pd.read_csv('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/reference_data/h5ad_metadata_tracker_121222.csv')
h5ad_to_modify = read_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/resegmented_data_copies/antibody_tech_reps_121222.h5ad')
pseudotime = pd.read_csv('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/reference_data/pseudotime.csv', index_col=0) #Brian's version


In [21]:
h5ad_tracker.columns
selected_rows = h5ad_tracker[['Donor', 'Specimen', 'Barcode', 'UWA ID','Specimen Type', 'Unique Donor ID','technical replicates', 'Dementia', 'Age (Y)', 'Sex', 'Final QC status']]
selected_rows['Barcode'] = selected_rows['Barcode'].apply(lambda x: x if pd.isnull(x) else str(int(x)))
selected_rows = selected_rows.rename(columns={'Barcode': 'filename', 'UWA ID': 'uwa', 'Specimen type': 'ADNC level'})
selected_rows = selected_rows.dropna(how='all')
selected_rows = selected_rows.drop_duplicates(subset='filename')
# from collections import Counter
# test = Counter(selected_rows.filename.tolist())
# dups = [x for x in selected_rows.filename.tolist() if test[x]>1]
selected_rows

/tmp/ipykernel_3193928/2872692647.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['Barcode'] = selected_rows['Barcode'].apply(lambda x: x if pd.isnull(x) else str(int(x)))


,Donor,Specimen,filename,uwa,Specimen Type,Unique Donor ID,technical replicates,Dementia,Age (Y),Sex,Final QC status
0,H20.33.044,H20.33.044.Cx26.MTG.02.007.1.01.02,1172983297,6962.0,Neurotypical,H20.33.044.01,H20.33.044_01,N,81.0,M,q
1,H20.33.044,H20.33.044.Cx26.MTG.02.007.1.01.03,1172983302,6962.0,Neurotypical,H20.33.044.02,H20.33.044_01,N,81.0,M,q
2,H20.33.044,H20.33.044.Cx26.MTG.02.007.1.01.04,1172983307,6962.0,Neurotypical,H20.33.044.03,H20.33.044_01,N,81.0,M,p
3,H19.33.004,H19.33.004.Cx22.MTG.02.007.1.01.02,NaN,NaN,Neurotypical,H19.33.004.01,NaN,N,80.0,F,f
8,H21.33.015,H21.33.015.Cx26.MTG.02.007.1.0,1162693722,7175.0,ADNC 1,H21.33.015.01,H21.33.015_01,N,98.0,M,f
...,...,...,...,...,...,...,...,...,...,...,...
107,H21.33.038.Cx20.MTG.02.007.3.01,H21.33.038.Cx20.MTG.02.007.3.01.03,1218552106,7323.0,ADNC 1,H21.33.038.02,H21.33.038_01,Y,84.0,F,f
108,H21.33.038.Cx20.MTG.02.007.3.01,H21.33.038.Cx20.MTG.02.007.3.01.04,1218552111,7323.0,ADNC 1,H21.33.038.03,H21.33.038_01,Y,84.0,F,q
109,H21.33.025.CX26.MTG.02.007.4.01,H21.33.025.CX26.MTG.02.007.4.01.02,1218552010,7244.0,ADNC 2,H21.33.025.01,H21.33.025_01,Y,88.0,F,q
110,H21.33.025.CX26.MTG.02.007.4.01,H21.33.025.CX26.MTG.02.007.4.01.04,1218552020,7244.0,ADNC 2,H21.33.025.02,H21.33.025_01,Y,88.0,F,q


In [22]:
pseudotime
selected_rows_pt = pd.merge(selected_rows, pseudotime, on='uwa', how='left')
selected_rows_pt['uwa'] = selected_rows_pt['uwa'].astype('Int64')
selected_rows_pt =  selected_rows_pt[selected_rows_pt['filename'].notna()]

In [23]:
#h5ad_to_modify.obs = h5ad_to_modify.obs.drop(['status', 'section'], axis=1)
h5ad_to_modify.obs.columns

Index(['cluster', 'subclass_scANVI', 'neighborhood', 'class', 'merscope',
       'avg.cor', 'genes_detected', 'total_reads', 'prob', 'volume', 'section',
       'filename'],
      dtype='object')

In [24]:
h5ad_with_metadata = pd.merge(h5ad_to_modify.obs, selected_rows_pt, on = 'filename', how='left')
print(len(h5ad_with_metadata), len(h5ad_to_modify.obs), len(selected_rows_pt))
# len(set(h5ad_with_metadata[h5ad_with_metadata.duplicated(subset=['filename'])].filename.values.tolist()))
#if extra _x, _y in h5ad_with_metadata.columns
# dropcols = [x for x in h5ad_with_metadata.columns if '_y' in x]
# h5ad_with_metadata = h5ad_with_metadata.drop(columns=dropcols)
# new_cols = [x[:-2] if '_x' in x else x for x in h5ad_with_metadata.columns]
# rename_dict = dict(zip(h5ad_with_metadata.columns, new_cols))
# h5ad_with_metadata = h5ad_with_metadata.rename(columns=rename_dict)
# h5ad_with_metadata.columns


97916 97916 78


In [25]:
h5ad_to_modify.obs = h5ad_with_metadata
h5ad_to_modify.write_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/resegmented_data_copies/antibody_tech_reps_121222.h5ad')

/home/imaging_mfish/anaconda3/envs/survey_nb/lib/python3.8/site-packages/anndata/_core/anndata.py:798: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)
